In [1]:
library(data.table)
library(tictoc)

clinical_dt <- fread("clinical_labs.csv")
lab_dt <- fread("lab_reference_ranges.csv")
meta_dt <- fread("sample_metadata.csv")

#avvio timer generale
tic("Esercizio5")

#il non-equi join, ti mette insieme specificando una disu. facendo: lower<= x <=upper

#TASK1
tic("task1")
ref_ranges <- lab_dt[, .(lower = first(lower), upper = first(upper)), by = lab] #crea

classified_labs <- clinical_dt[ref_ranges, on = "lab"]

classified_labs[, status := fifelse( # fifelse= valutazione condizionale,più veloce
    value >= lower & value <= upper,
    "normal",
    "out_of_range"
)]

abnormal_rates <- classified_labs[,
    .(
        n_total = .N,
        n_abnormal = sum(status == "out_of_range"),
        abnormal_rate = sum(status == "out_of_range") / .N
    ),
    by = .(patient_id, lab)
][order(-abnormal_rate)] # Ordina per tasso decrescente

#fine timer task1
toc()

# Stampa i risultati
cat("\n\n Classificazione dei primi risultati\n")
print(head(classified_labs[, .(patient_id, lab, value, lower, upper, status)]))

cat("\n\n        Tassi Anomali\n")
print(head(abnormal_rates, 10))

#TASK2
tic("task2")

abnormal_rates <- classified_labs[,
    .(
        # 1. Conteggio totale per gruppo (.N)
        n_total = .N,
        # 2. Conteggio dei risultati fuori range
        n_abnormal = sum(status == "out_of_range"),
        # 3. Calcolo del tasso di anomalia (abnormal / total)
        abnormal_rate = sum(status == "out_of_range") / .N
    ),
    by = .(patient_id, lab)
][order(-abnormal_rate)] # Ordina per tasso decrescente

toc()
print(head(abnormal_rates, 10))

#fine timer generale
toc()


Attaching package: ‘tictoc’


The following object is masked from ‘package:data.table’:

    shift




task1: 0.024 sec elapsed


 Classificazione dei primi risultati
   patient_id    lab     value lower upper       status
       <char> <char>     <num> <num> <num>       <char>
1:       P002    CRP  0.100000     0     5       normal
2:       P002    CRP  9.709428     0     5 out_of_range
3:       P002    CRP  6.359407     0     5 out_of_range
4:       P002    CRP  0.100000     0     5       normal
5:       P002    CRP  5.632359     0     5 out_of_range
6:       P002    CRP 10.248945     0     5 out_of_range


        Tassi Anomali
    patient_id    lab n_total n_abnormal abnormal_rate
        <char> <char>   <int>      <int>         <num>
 1:       P007    CRP       8          6     0.7500000
 2:       P018    CRP       7          5     0.7142857
 3:       P002    CRP       6          4     0.6666667
 4:       P021    CRP       6          4     0.6666667
 5:       P053    CRP      10          6     0.6000000
 6:       P036    CRP       7          4     0.5714286
 7:       P038    CRP   

In [2]:
library(data.table)
library(tictoc)

# Caricamento dati (omesso per brevità)
clinical_dt <- fread("clinical_labs.csv")
lab_dt <- fread("lab_reference_ranges.csv")

# Avvio timer generale
tic("Esercizio Completo") 

# ====================================================================
# TASK 1: CLASSIFICAZIONE (Produzione della colonna status)
# ====================================================================
tic("Task 1: Classificazione")

ref_ranges <- lab_dt[, .(lower = first(lower), upper = first(upper)), by = lab]
classified_labs <- clinical_dt[ref_ranges, on = "lab"]

classified_labs[, status := fifelse(
    value >= lower & value <= upper,
    "normal",
    "out_of_range"
)]

toc() # Fine timer Task 1

# ====================================================================
# TASK 2: CONTEGGIO E TASSI (Aggregazione)
# ====================================================================
tic("Task 2: Conteggio Tassi")

abnormal_rates <- classified_labs[,
    .(
        n_total = .N,
        n_abnormal = sum(status == "out_of_range"),
        abnormal_rate = sum(status == "out_of_range") / .N
    ),
    by = .(patient_id, lab)
][order(-abnormal_rate)] 

toc() # Fine timer Task 2

# ====================================================================
# STAMPA RISULTATI E FINE
# ====================================================================
# Fine timer generale
toc() 

cat("\n\n----------------------------------")
cat("\n Risultati Task 1 (Classificazione) \n")
cat("----------------------------------\n")
print(head(classified_labs[, .(patient_id, lab, value, lower, upper, status)]))

cat("\n\n------------------------------")
cat("\n Risultati Task 2 (Tassi Anomali) \n")
cat("------------------------------\n")
print(head(abnormal_rates, 10))

Task 1: Classificazione: 0.009 sec elapsed
Task 2: Conteggio Tassi: 0.002 sec elapsed
Esercizio Completo: 0.017 sec elapsed


----------------------------------
 Risultati Task 1 (Classificazione) 
----------------------------------
   patient_id    lab     value lower upper       status
       <char> <char>     <num> <num> <num>       <char>
1:       P002    CRP  0.100000     0     5       normal
2:       P002    CRP  9.709428     0     5 out_of_range
3:       P002    CRP  6.359407     0     5 out_of_range
4:       P002    CRP  0.100000     0     5       normal
5:       P002    CRP  5.632359     0     5 out_of_range
6:       P002    CRP 10.248945     0     5 out_of_range


------------------------------
 Risultati Task 2 (Tassi Anomali) 
------------------------------
    patient_id    lab n_total n_abnormal abnormal_rate
        <char> <char>   <int>      <int>         <num>
 1:       P007    CRP       8          6     0.7500000
 2:       P018    CRP       7          5     0.7142857


In [3]:
library(tictoc)

# Caricamento dei set di dati come data.frame (usando read.csv)
clinical_df <- read.csv("clinical_labs.csv")
lab_df <- read.csv("lab_reference_ranges.csv")
meta_df <- read.csv("sample_metadata.csv")

# Avvio timer generale
tic("Esercizio 5 data.frame") 

# TASK1
tic("Task1")

# 2. Preparazione dei Range di Riferimento
# Usiamo aggregate per semplificare i range, ottenendo un unico intervallo per 'lab'.
ref_ranges <- unique(lab_df[c("lab", "lower", "upper")])

# 3. Join
# Utilizziamo merge (l'equivalente di join in data.frame)
classified_labs <- merge(clinical_df, ref_ranges, by = "lab", all.x = TRUE)

# 4. Classificazione
# Utilizziamo ifelse (l'equivalente di fifelse) con logica vettoriale
classified_labs$status <- ifelse(
    classified_labs$value >= classified_labs$lower & classified_labs$value <= classified_labs$upper,
    "normal",
    "out_of_range"
)

# Fine timer Task 1
toc()

#TASK2

tic("Task2")

# 5. Calcolo dei tassi anomali
# Si usa aggregate (funzione base R) per raggruppare (by) e calcolare le statistiche.
abnormal_rates_df <- aggregate(
    x = classified_labs$status,
    by = list(patient_id = classified_labs$patient_id, lab = classified_labs$lab),
    FUN = function(x) {
        n_total = length(x)
        n_abnormal = sum(x == "out_of_range")
        # Restituisce un vettore con i tre valori
        c(n_total = n_total, n_abnormal = n_abnormal, abnormal_rate = n_abnormal / n_total)
    }
)

# Ristrutturazione del risultato di aggregate (che è complesso in base R)
abnormal_rates_df <- data.frame(
    patient_id = abnormal_rates_df$patient_id,
    lab = abnormal_rates_df$lab,
    n_total = abnormal_rates_df$x[, "n_total"],
    n_abnormal = abnormal_rates_df$x[, "n_abnormal"],
    abnormal_rate = abnormal_rates_df$x[, "abnormal_rate"]
)

# 6. Ordinamento
# Si usa order per ordinare in base al tasso anomalo decrescente
abnormal_rates_df <- abnormal_rates_df[order(-abnormal_rates_df$abnormal_rate), ]

toc()
# Fine timer Task 2

# Fine timer generale
run_time <- toc() 

cat("\n  Classificazione dei primi valori \n")
print(head(classified_labs[c("patient_id", "lab", "value", "lower", "upper", "status")]))
cat("\n Tassi Anomali  \n")

print(head(abnormal_rates_df, 10))

cat(sprintf("\n\nTempo di esecuzione Totale (tictoc): %.4f secondi\n", run_time$toc - run_time$tic))

Task1: 0.003 sec elapsed
Task2: 0.009 sec elapsed
Esercizio 5 data.frame: 0.016 sec elapsed

  Classificazione dei primi valori 
  patient_id lab    value lower upper       status
1       P002 CRP 0.100000     0     5       normal
2       P007 CRP 8.307172     0     5 out_of_range
3       P002 CRP 9.709428     0     5 out_of_range
4       P007 CRP 7.601454     0     5 out_of_range
5       P002 CRP 6.359407     0     5 out_of_range
6       P007 CRP 7.757868     0     5 out_of_range

 Tassi Anomali  
   patient_id lab n_total n_abnormal abnormal_rate
2        P007 CRP       8          6     0.7500000
9        P018 CRP       7          5     0.7142857
1        P002 CRP       6          4     0.6666667
11       P021 CRP       6          4     0.6666667
19       P053 CRP      10          6     0.6000000
15       P036 CRP       7          4     0.5714286
16       P038 CRP       9          5     0.5555556
6        P013 CRP       6          3     0.5000000
17       P044 CRP      10          5 

In [4]:
#tabella confronto tempi
T_DT1 <- 0.009
T_DT2 <- 0.002
T_DF1 <- 0.003
T_DF2 <- 0.009

# Creazione della tabella riassuntiva
risultati_performance <- data.frame(
  Task = c("Task 1",
           "Task 2"),
  
  Tempo_data.table_Sec = c(T_DT1, T_DT2),
  Tempo_data.frame_Sec = c(T_DF1, T_DF2)
)

# Aggiungiamo una colonna per il fattore di velocizzazione (Speedup)
risultati_performance$Speedup_DT_vs_DF <- 
  round(risultati_performance$Tempo_data.frame_Sec / risultati_performance$Tempo_data.table_Sec, 1)

# Stampiamo la tabella finale
print(risultati_performance)

    Task Tempo_data.table_Sec Tempo_data.frame_Sec Speedup_DT_vs_DF
1 Task 1                0.009                0.003              0.3
2 Task 2                0.002                0.009              4.5
